# Gold vs. NIFTY 50 (TR)

Combines data from the FRED and NSE to study gold and NIFTY 50 TR returns.

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
options("scipen"=999)

source("goofy/misc.common.R")
source("goofy/plot.common.R")
source("config.R")

fred<-Fred()
indices<-Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


ERROR: Error in library(ggthemes): there is no package called ‘ggthemes’


## Gold
first, lets check what kind of data is available on the FRED for gold

In [2]:
goldFred <- fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%gold%') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS)

print(data.frame(goldFred))

ERROR: Error in eval(lhs, parent, parent): object 'fred' not found


In [3]:
seriesIdGold <- -2147252004 #use the afternoon USD fix
goldDf <- data.frame(fred$TimeSeries() %>% filter(SERIES_ID == seriesIdGold, TIME_STAMP >= as.Date('1995-01-01')) %>% 
    select(TIME_STAMP, VAL))
goldDf$TIME_STAMP <- as.Date(goldDf$TIME_STAMP)

fredDetailsGold <- data.frame(goldFred %>% filter(SERIES_ID == seriesIdGold))

print(nrow(goldDf))
print(head(goldDf))

goldXts <- xts(goldDf$VAL, goldDf$TIME_STAMP)

ERROR: Error in eval(lhs, parent, parent): object 'fred' not found


### plot the price of gold in USD

In [4]:
firstDate <- min(goldDf$TIME_STAMP)
lastDate <- max(goldDf$TIME_STAMP)
xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=10)

options(repr.plot.width=16, repr.plot.height=8)

ggplot(goldDf, aes(x=TIME_STAMP, y=VAL)) +
    theme_economist() +
    #theme(text = element_text(size = 3), element_line(size = 0.1)) +
    geom_line() +
    scale_y_log10() +
    scale_x_date(breaks = xAxisTicks) +
    labs(x='', y='log(price)', color='', title=sprintf("FRED: %s - %s", fredDetailsGold$TICKER[1], fredDetailsGold$NAME[1]), 
         subtitle=sprintf("[%s:%s]", firstDate, lastDate)) +
    annotate("text", x=lastDate, y=min(goldDf$VAL, na.rm=T), label = "@StockViz", hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)

ERROR: Error in eval(expr, envir, enclos): object 'goldDf' not found


## NIFTY 50

The NIFTY 50 TR index has been around since 1999. Translate that index to USD and plot monthly returns vs. gold.

In [5]:
trIndex <- data.frame(indices$NseTimeSeries() %>%
    filter(NAME == "NIFTY 50 TR" && TIME_STAMP >= '2000-01-01' && TIME_STAMP <= lastDate) %>%
    select(TIME_STAMP, CLOSE))

trXts <- xts(trIndex$CLOSE, as.Date(trIndex$TIME_STAMP))

ERROR: Error in eval(lhs, parent, parent): object 'indices' not found


In [6]:
## USDINR

In [7]:
#DEXINUS, from the example: http://pluto.studio/user/shyams80/notebooks/docs-R/Fred.ipynb
seriesIdUsdInr <- -2147478748
usdInrDf <- data.frame(fred$TimeSeries() %>% filter(SERIES_ID == seriesIdUsdInr, TIME_STAMP >= as.Date('2000-01-01')) %>% 
    select(TIME_STAMP, VAL))
usdInrDf$TIME_STAMP <- as.Date(usdInrDf$TIME_STAMP)

fredDetailsUsdInr <- data.frame(fred$Meta() %>% filter(SERIES_ID == seriesIdUsdInr))

print(nrow(usdInrDf))
print(head(usdInrDf))

usdInrXts <- xts(usdInrDf$VAL, usdInrDf$TIME_STAMP)

ERROR: Error in eval(lhs, parent, parent): object 'fred' not found


## Merge, compute and plot returns

In [8]:
allXts <- merge(trXts, usdInrXts, goldXts)
#holidays don't match up, so carry the USDINR forward.
allXts[,2] <- na.locf(allXts[,2])
allXts <- na.omit(allXts)

ERROR: Error in merge(trXts, usdInrXts, goldXts): object 'trXts' not found


In [9]:
#end-of months don't match, so force them to a common on
monthlies <- merge(Common.NormalizeMonthlyDates(monthlyReturn(allXts[,1]/allXts[,2])), 
                   Common.NormalizeMonthlyDates(monthlyReturn(allXts[,3])))
names(monthlies) <-c('TR', 'GOLD')

print(head(monthlies))

ERROR: Error in Common.NormalizeMonthlyDates(monthlyReturn(allXts[, 1]/allXts[, : could not find function "Common.NormalizeMonthlyDates"


In [10]:
Common.PlotCumReturns(monthlies, "NIFTY 50 TR vs. Gold", "in USD")

ERROR: Error in Common.PlotCumReturns(monthlies, "NIFTY 50 TR vs. Gold", "in USD"): could not find function "Common.PlotCumReturns"


## Take-away

NIFTY 50 has  point over gold in annualized returns. Better yet, the asset classes seem to be uncorrelated so they diversify each other.

For people with a time horizon of 8+ years, the RBI issued sovereign gold bonds are a good option. 

Related: https://stockviz.biz/2016/12/02/investing-in-gold/

*This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)*